In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

# EDA

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Competition/Home Credit Default Risk/application_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Competition/Home Credit Default Risk/application_test.csv')
bureau = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Competition/Home Credit Default Risk/bureau.csv')
bureau_balance = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Competition/Home Credit Default Risk/bureau_balance.csv')
credit_card_balance = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Competition/Home Credit Default Risk/credit_card_balance.csv')
install_payments = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Competition/Home Credit Default Risk/installments_payments.csv')
pos_cash_balance = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Competition/Home Credit Default Risk/POS_CASH_balance.csv')
previous_application = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Competition/Home Credit Default Risk/previous_application.csv')


In [ ]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train.shape

(307511, 122)

In [ ]:
# Absolute correlation of TARGET with numeric features
corr_target_abs = (
    train.corr(numeric_only=True)['TARGET']
    .drop('TARGET')                           # remove self-correlation
    .abs()                                    # take absolute values
    .sort_values(ascending=True)              # sort ascending
)

print(corr_target_abs)


FLAG_DOCUMENT_20               0.000215
FLAG_DOCUMENT_5                0.000316
FLAG_CONT_MOBILE               0.000370
FLAG_MOBIL                     0.000534
FLAG_DOCUMENT_12               0.000756
                                 ...   
REGION_RATING_CLIENT_W_CITY    0.060893
DAYS_BIRTH                     0.078239
EXT_SOURCE_1                   0.155317
EXT_SOURCE_2                   0.160472
EXT_SOURCE_3                   0.178919
Name: TARGET, Length: 105, dtype: float64


In [ ]:
# correlations with TARGET
corr_target = (
    train.corr(numeric_only=True)['TARGET']
    .drop('TARGET')
)

# filter by absolute correlation < 0.05
low_corr_features = corr_target[ corr_target.abs() < 0.05 ].index.tolist()

print("Number of numeric features with |corr| < 0.05:", len(low_corr_features))
print(low_corr_features)


Number of numeric features with |corr| < 0.05: 96
['SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE',

In [ ]:
train['TARGET'].value_counts(
    normalize=True
)

,proportion
TARGET,
0,0.919271
1,0.080729


In [ ]:
missing = train.isnull().mean().sort_values(ascending=False) * 100
print(missing.head(20))  # top 20 features with most NaNs


COMMONAREA_AVG              69.872297
COMMONAREA_MODE             69.872297
COMMONAREA_MEDI             69.872297
NONLIVINGAPARTMENTS_MEDI    69.432963
NONLIVINGAPARTMENTS_MODE    69.432963
NONLIVINGAPARTMENTS_AVG     69.432963
FONDKAPREMONT_MODE          68.386172
LIVINGAPARTMENTS_AVG        68.354953
LIVINGAPARTMENTS_MEDI       68.354953
LIVINGAPARTMENTS_MODE       68.354953
FLOORSMIN_MODE              67.848630
FLOORSMIN_AVG               67.848630
FLOORSMIN_MEDI              67.848630
YEARS_BUILD_AVG             66.497784
YEARS_BUILD_MODE            66.497784
YEARS_BUILD_MEDI            66.497784
OWN_CAR_AGE                 65.990810
LANDAREA_MEDI               59.376738
LANDAREA_AVG                59.376738
LANDAREA_MODE               59.376738
dtype: float64


In [ ]:
# define categorical (object) columns
object_cols = train.select_dtypes(include=['object']).columns

unique_counts_df = (
    train[object_cols]
    .nunique()
    .reset_index()
    .rename(columns={'index': 'Feature', 0: 'UniqueValues'})
    .sort_values(by='UniqueValues')
)

print(unique_counts_df)


                       Feature  UniqueValues
0           NAME_CONTRACT_TYPE             2
2                 FLAG_OWN_CAR             2
3              FLAG_OWN_REALTY             2
15         EMERGENCYSTATE_MODE             2
13              HOUSETYPE_MODE             3
1                  CODE_GENDER             3
12          FONDKAPREMONT_MODE             4
6          NAME_EDUCATION_TYPE             5
7           NAME_FAMILY_STATUS             6
8            NAME_HOUSING_TYPE             6
10  WEEKDAY_APPR_PROCESS_START             7
4              NAME_TYPE_SUITE             7
14          WALLSMATERIAL_MODE             7
5             NAME_INCOME_TYPE             8
9              OCCUPATION_TYPE            18
11           ORGANIZATION_TYPE            58


# Removing NaN values

In [ ]:
# % of NaNs per column
nan_percent = train.isnull().mean() * 100

# filter columns with >50% NaN
cols_over_50_nan = nan_percent[nan_percent > 50].index.tolist()

print("Number of columns with >50% NaN:", len(cols_over_50_nan))
print(cols_over_50_nan)


Number of columns with >50% NaN: 41
['OWN_CAR_AGE', 'EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE']


In [ ]:
cols_over_50_nan = ['OWN_CAR_AGE', 'EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BUILD_AVG',
                    'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG',
                    'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE',
                    'BASEMENTAREA_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMIN_MODE',
                    'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE',
                    'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI',
                    'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI',
                    'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE']

# Split Data

In [ ]:
X = train.drop(['TARGET'], axis=1)
y = train['TARGET']

In [ ]:
X.drop(cols_over_50_nan, axis=1, inplace=True)

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 80 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   SK_ID_CURR                    307511 non-null  int64  
 1   NAME_CONTRACT_TYPE            307511 non-null  object 
 2   CODE_GENDER                   307511 non-null  object 
 3   FLAG_OWN_CAR                  307511 non-null  object 
 4   FLAG_OWN_REALTY               307511 non-null  object 
 5   CNT_CHILDREN                  307511 non-null  int64  
 6   AMT_INCOME_TOTAL              307511 non-null  float64
 7   AMT_CREDIT                    307511 non-null  float64
 8   AMT_ANNUITY                   307499 non-null  float64
 9   AMT_GOODS_PRICE               307233 non-null  float64
 10  NAME_TYPE_SUITE               306219 non-null  object 
 11  NAME_INCOME_TYPE              307511 non-null  object 
 12  NAME_EDUCATION_TYPE           307511 non-nul

In [ ]:
for col in X.columns:
    if X[col].isna().sum()>0:
        print(f'{col}: {X[col].isna().sum()}')

AMT_ANNUITY: 12
AMT_GOODS_PRICE: 278
NAME_TYPE_SUITE: 1292
OCCUPATION_TYPE: 96391
CNT_FAM_MEMBERS: 2
EXT_SOURCE_2: 660
EXT_SOURCE_3: 60965
YEARS_BEGINEXPLUATATION_AVG: 150007
FLOORSMAX_AVG: 153020
YEARS_BEGINEXPLUATATION_MODE: 150007
FLOORSMAX_MODE: 153020
YEARS_BEGINEXPLUATATION_MEDI: 150007
FLOORSMAX_MEDI: 153020
TOTALAREA_MODE: 148431
EMERGENCYSTATE_MODE: 145755
OBS_30_CNT_SOCIAL_CIRCLE: 1021
DEF_30_CNT_SOCIAL_CIRCLE: 1021
OBS_60_CNT_SOCIAL_CIRCLE: 1021
DEF_60_CNT_SOCIAL_CIRCLE: 1021
DAYS_LAST_PHONE_CHANGE: 1
AMT_REQ_CREDIT_BUREAU_HOUR: 41519
AMT_REQ_CREDIT_BUREAU_DAY: 41519
AMT_REQ_CREDIT_BUREAU_WEEK: 41519
AMT_REQ_CREDIT_BUREAU_MON: 41519
AMT_REQ_CREDIT_BUREAU_QRT: 41519
AMT_REQ_CREDIT_BUREAU_YEAR: 41519


# Filling NaN values

In [ ]:
X['AMT_ANNUITY'].fillna(X['AMT_ANNUITY'].mean(), inplace=True)
X['AMT_GOODS_PRICE'].fillna(X['AMT_GOODS_PRICE'].mean(), inplace=True)
X['NAME_TYPE_SUITE'].fillna('Unidentified', inplace=True)
X['OCCUPATION_TYPE'].fillna('Unemployed', inplace=True)
X['CNT_FAM_MEMBERS'].fillna(X['CNT_FAM_MEMBERS'].mean(), inplace=True)
X['EXT_SOURCE_2'].fillna(X['EXT_SOURCE_2'].mean(), inplace=True)
X['EXT_SOURCE_3'].fillna(X['EXT_SOURCE_3'].mean(), inplace=True)
X['YEARS_BEGINEXPLUATATION_AVG'].fillna(X['YEARS_BEGINEXPLUATATION_AVG'].mean(), inplace=True)
X['FLOORSMAX_AVG'].fillna(X['FLOORSMAX_AVG'].mean(), inplace=True)
X['YEARS_BEGINEXPLUATATION_MODE'].fillna(X['YEARS_BEGINEXPLUATATION_MODE'].mean(), inplace=True)
X['FLOORSMAX_MODE'].fillna(X['FLOORSMAX_MODE'].mean(), inplace=True)
X['YEARS_BEGINEXPLUATATION_MEDI'].fillna(X['YEARS_BEGINEXPLUATATION_MEDI'].mean(), inplace=True)
X['FLOORSMAX_MEDI'].fillna(X['FLOORSMAX_MEDI'].mean(), inplace=True)
X['TOTALAREA_MODE'].fillna(X['TOTALAREA_MODE'].mean(), inplace=True)
X['EMERGENCYSTATE_MODE'].fillna('No', inplace=True)
X['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(X['OBS_30_CNT_SOCIAL_CIRCLE'].mode()[0], inplace=True)
X['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(X['DEF_30_CNT_SOCIAL_CIRCLE'].mode()[0], inplace=True)
X['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(X['OBS_60_CNT_SOCIAL_CIRCLE'].mode()[0], inplace=True)
X['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(X['DEF_60_CNT_SOCIAL_CIRCLE'].mode()[0], inplace=True)
X['DAYS_LAST_PHONE_CHANGE'].fillna(X['DAYS_LAST_PHONE_CHANGE'].mode()[0], inplace=True)
X['AMT_REQ_CREDIT_BUREAU_HOUR'].fillna(X['AMT_REQ_CREDIT_BUREAU_HOUR'].mode()[0], inplace=True)
X['AMT_REQ_CREDIT_BUREAU_DAY'].fillna(X['AMT_REQ_CREDIT_BUREAU_DAY'].mode()[0], inplace=True)
X['AMT_REQ_CREDIT_BUREAU_WEEK'].fillna(X['AMT_REQ_CREDIT_BUREAU_WEEK'].mode()[0], inplace=True)
X['AMT_REQ_CREDIT_BUREAU_MON'].fillna(X['AMT_REQ_CREDIT_BUREAU_MON'].mode()[0], inplace=True)
X['AMT_REQ_CREDIT_BUREAU_QRT'].fillna(X['AMT_REQ_CREDIT_BUREAU_QRT'].mode()[0], inplace=True)
X['AMT_REQ_CREDIT_BUREAU_YEAR'].fillna(X['AMT_REQ_CREDIT_BUREAU_YEAR'].mode()[0], inplace=True)


/tmp/ipython-input-3154249824.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['AMT_ANNUITY'].fillna(X['AMT_ANNUITY'].mean(), inplace=True)
/tmp/ipython-input-3154249824.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 80 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   SK_ID_CURR                    307511 non-null  int64  
 1   NAME_CONTRACT_TYPE            307511 non-null  object 
 2   CODE_GENDER                   307511 non-null  object 
 3   FLAG_OWN_CAR                  307511 non-null  object 
 4   FLAG_OWN_REALTY               307511 non-null  object 
 5   CNT_CHILDREN                  307511 non-null  int64  
 6   AMT_INCOME_TOTAL              307511 non-null  float64
 7   AMT_CREDIT                    307511 non-null  float64
 8   AMT_ANNUITY                   307511 non-null  float64
 9   AMT_GOODS_PRICE               307511 non-null  float64
 10  NAME_TYPE_SUITE               307511 non-null  object 
 11  NAME_INCOME_TYPE              307511 non-null  object 
 12  NAME_EDUCATION_TYPE           307511 non-nul

In [ ]:
# Select object (categorical) columns
object_cols = X.select_dtypes(include='object')

# Get number of unique values for each
obj_unique_counts = object_cols.nunique().sort_values()

print(obj_unique_counts)


NAME_CONTRACT_TYPE             2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
EMERGENCYSTATE_MODE            2
CODE_GENDER                    3
NAME_EDUCATION_TYPE            5
NAME_HOUSING_TYPE              6
NAME_FAMILY_STATUS             6
WEEKDAY_APPR_PROCESS_START     7
NAME_INCOME_TYPE               8
NAME_TYPE_SUITE                8
OCCUPATION_TYPE               19
ORGANIZATION_TYPE             58
dtype: int64


In [ ]:
X['CODE_GENDER'] = X['CODE_GENDER'].replace('XNA', 'F')
X['NAME_EDUCATION_TYPE'] = X['NAME_EDUCATION_TYPE'].map({'Secondary / secondary special': 1, 'Lower secondary': 1,
                                                         'Higher education': 2,
                                                         'Incomplete higher': 2,
                                                         'Academic degree': 2})
X['NAME_FAMILY_STATUS'] = X['NAME_FAMILY_STATUS'].map({'Married': 1, 'Civil marriage': 1,
                                                       'Single / not married':2,
                                                       'Separated':2, 'Widow':2, 'Unknown':2})
X['NAME_INCOME_TYPE'] = X['NAME_INCOME_TYPE'].map({'Working':1, 'Commercial associate':1, 'Pensioner':2,
                                                   'State servant':2, 'Unemployed':3,
                                                   'Student':3, 'Maternity leave':3,
                                                   'Businessman':0})

X['NAME_TYPE_SUITE'] = X['NAME_TYPE_SUITE'].map({'Unaccompanied':0, 'Family':1, 'Spouse, partner':2,
                                                 'Children':1, 'Other_B':3, 'Other_A':3, 'Unidentified':3,
                                                 'Group of people':3})
X['OCCUPATION_TYPE'] = X['OCCUPATION_TYPE'].map({
    'Unemployed': 0,
    'Low-skill Laborers': 1,
    'Cleaning staff': 1,
    'Cooking staff': 1,
    'Waiters/barmen staff': 1,
    'Private service staff': 1,

    'Laborers': 2,
    'Sales staff': 2,
    'Security staff': 2,
    'Drivers': 2,

    'Core staff': 3,
    'Medicine staff': 3,
    'Secretaries': 3,

    'Accountants': 4,
    'HR staff': 4,
    'Realty agents': 4,

    'Managers': 5,
    'High skill tech staff': 5,
    'IT staff': 5
})

X['ORGANIZATION_TYPE'] = X['ORGANIZATION_TYPE'].map({
    # 0 → Very low/unstable income, or undefined
    'XNA': 0,
    'Other': 0,
    'Religion': 0,
    'Kindergarten': 0,
    'School': 0,
    'Culture': 0,
    'Cleaning': 0,

    # 1 → Low-income service / manual work
    'Services': 1,
    'Restaurant': 1,
    'Hotel': 1,
    'Trade: type 1': 1,
    'Trade: type 2': 1,
    'Trade: type 3': 1,
    'Trade: type 4': 1,
    'Trade: type 5': 1,
    'Trade: type 6': 1,
    'Trade: type 7': 1,

    # 2 → Agriculture / simple industry
    'Agriculture': 2,
    'Industry: type 1': 2,
    'Industry: type 2': 2,
    'Industry: type 3': 2,
    'Industry: type 4': 2,
    'Industry: type 5': 2,
    'Industry: type 6': 2,
    'Industry: type 7': 2,
    'Industry: type 8': 2,
    'Industry: type 9': 2,
    'Industry: type 10': 2,
    'Industry: type 11': 2,
    'Industry: type 12': 2,
    'Industry: type 13': 2,
    'Construction': 2,
    'Housing': 2,

    # 3 → Transport, utilities, security
    'Transport: type 1': 3,
    'Transport: type 2': 3,
    'Transport: type 3': 3,
    'Transport: type 4': 3,
    'Electricity': 3,
    'Security': 3,
    'Postal': 3,

    # 4 → State institutions / government
    'Government': 4,
    'Military': 4,
    'Police': 4,
    'Security Ministries': 4,
    'Emergency': 4,
    'University': 4,

    # 5 → Self-employed / small businesses
    'Self-employed': 5,
    'Business Entity Type 1': 5,
    'Business Entity Type 2': 5,
    'Business Entity Type 3': 5,
    'Realtor': 5,

    # 6 → Professional services / white-collar
    'Medicine': 6,
    'Legal Services': 6,
    'Advertising': 6,
    'Mobile': 6,
    'Telecom': 6,
    'Insurance': 6,
    'Bank': 6
})

In [ ]:
# Select object (categorical) columns
object_cols = X.select_dtypes(include='object')

# Get number of unique values for each
obj_unique_counts = object_cols.nunique().sort_values()

print(obj_unique_counts)

NAME_CONTRACT_TYPE            2
CODE_GENDER                   2
FLAG_OWN_CAR                  2
FLAG_OWN_REALTY               2
EMERGENCYSTATE_MODE           2
NAME_HOUSING_TYPE             6
WEEKDAY_APPR_PROCESS_START    7
dtype: int64


In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_HOUSING_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'EMERGENCYSTATE_MODE']

encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# fit & transform
encoded_array = encoder.fit_transform(X[one_hot])

# get new column names
encoded_cols = encoder.get_feature_names_out(one_hot)

# convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoded_cols, index=X.index)

# concat with original DataFrame (dropping original categorical cols)
X = pd.concat([X.drop(columns=one_hot), encoded_df], axis=1)

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 89 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   SK_ID_CURR                             307511 non-null  int64  
 1   CNT_CHILDREN                           307511 non-null  int64  
 2   AMT_INCOME_TOTAL                       307511 non-null  float64
 3   AMT_CREDIT                             307511 non-null  float64
 4   AMT_ANNUITY                            307511 non-null  float64
 5   AMT_GOODS_PRICE                        307511 non-null  float64
 6   NAME_TYPE_SUITE                        307511 non-null  int64  
 7   NAME_INCOME_TYPE                       307511 non-null  int64  
 8   NAME_EDUCATION_TYPE                    307511 non-null  int64  
 9   NAME_FAMILY_STATUS                     307511 non-null  int64  
 10  REGION_POPULATION_RELATIVE             307511 non-null  

# Other datasets

In [ ]:
import pandas as pd

# compute mode of STATUS per SK_ID_BUREAU
status_mode = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].agg(lambda x: x.mode()[0])

# convert to DataFrame (optional)
status_mode = status_mode.reset_index().rename(columns={'STATUS':'STATUS_MODE'})

print(status_mode.head())


   SK_ID_BUREAU STATUS_MODE
0       5001709           C
1       5001710           C
2       5001711           0
3       5001712           0
4       5001713           X


In [ ]:
bureau_balance = status_mode.copy()

In [ ]:
bureau_balance.shape

(817395, 2)

In [ ]:
bureau_balance['SK_ID_BUREAU'].nunique()

817395

In [ ]:
bureau['SK_ID_BUREAU'].nunique()

1716428

In [ ]:
bureau_total = bureau.merge(bureau_balance, on='SK_ID_BUREAU', how='left')
bureau_total.shape

(1716428, 18)

In [ ]:
bureau_total[bureau_total['SK_ID_CURR']==100001]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,STATUS_MODE
248484,100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,NaN,0,112500.0,0.0,0.0,0.0,Consumer credit,-155,0.0,C
248485,100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,NaN,0,279720.0,0.0,0.0,0.0,Consumer credit,-155,0.0,C
248486,100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,NaN,0,91620.0,0.0,0.0,0.0,Consumer credit,-155,0.0,C
248487,100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,NaN,0,85500.0,0.0,0.0,0.0,Consumer credit,-155,0.0,C
248488,100001,5896634,Active,currency 1,-559,0,902.0,NaN,NaN,0,337680.0,113166.0,0.0,0.0,Consumer credit,-6,4630.5,0
248489,100001,5896635,Active,currency 1,-49,0,1778.0,NaN,NaN,0,378000.0,373239.0,0.0,0.0,Consumer credit,-16,10822.5,0
248490,100001,5896636,Active,currency 1,-320,0,411.0,NaN,NaN,0,168345.0,110281.5,NaN,0.0,Consumer credit,-10,9364.5,0


In [ ]:
import pandas as pd

# Example safe_mode function
def safe_mode(s):
    m = s.mode()
    if len(m) == 0:
        return pd.NA  # or default value
    else:
        return m[0]

# Separate numeric and object columns
numeric_cols = bureau_total.select_dtypes(include='number').columns
object_cols = bureau_total.select_dtypes(include='object').columns

# Create aggregation dictionary
agg_dict = {col: 'mean' for col in numeric_cols}
agg_dict.update({col: safe_mode for col in object_cols})

# Group by SK_ID_CURR and aggregate
bureau_agg = bureau_total.groupby('SK_ID_CURR', as_index=False).agg(agg_dict)

# View result
print(bureau_agg.head())


   SK_ID_CURR  SK_ID_BUREAU  DAYS_CREDIT  CREDIT_DAY_OVERDUE  \
0    100001.0   5896633.000  -735.000000                 0.0   
1    100002.0   6153272.125  -874.000000                 0.0   
2    100003.0   5885878.500 -1400.750000                 0.0   
3    100004.0   6829133.500  -867.000000                 0.0   
4    100005.0   6735201.000  -190.666667                 0.0   

   DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  AMT_CREDIT_MAX_OVERDUE  \
0            82.428571        -825.500000                     NaN   
1          -349.000000        -697.500000                1681.029   
2          -544.500000       -1097.333333                   0.000   
3          -488.500000        -532.500000                   0.000   
4           439.333333        -123.000000                   0.000   

   CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  AMT_CREDIT_SUM_DEBT  \
0                 0.0   207623.571429         85240.928571   
1                 0.0   108131.945625         49156.200000   
2             

In [ ]:
bureau_agg['SK_ID_CURR'] = bureau_agg['SK_ID_CURR'].astype(int)
bureau_agg

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CREDIT_ACTIVE,CREDIT_CURRENCY,CREDIT_TYPE,STATUS_MODE
0,100001,5896633.000,-735.000000,0.0,82.428571,-825.500000,NaN,0.000000,2.076236e+05,85240.928571,0.00000,0.0,-93.142857,3545.357143,Closed,currency 1,Consumer credit,C
1,100002,6153272.125,-874.000000,0.0,-349.000000,-697.500000,1681.029,0.000000,1.081319e+05,49156.200000,7997.14125,0.0,-499.875000,0.000000,Closed,currency 1,Consumer credit,1
2,100003,5885878.500,-1400.750000,0.0,-544.500000,-1097.333333,0.000,0.000000,2.543501e+05,0.000000,202500.00000,0.0,-816.000000,NaN,Closed,currency 1,Consumer credit,<NA>
3,100004,6829133.500,-867.000000,0.0,-488.500000,-532.500000,0.000,0.000000,9.451890e+04,0.000000,0.00000,0.0,-532.000000,NaN,Closed,currency 1,Consumer credit,<NA>
4,100005,6735201.000,-190.666667,0.0,439.333333,-123.000000,0.000,0.000000,2.190420e+05,189469.500000,0.00000,0.0,-54.333333,1420.500000,Active,currency 1,Consumer credit,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,456249,5371700.000,-1667.076923,0.0,-1232.333333,-1364.750000,6147.000,0.000000,2.841430e+05,16307.100000,0.00000,0.0,-1064.538462,NaN,Closed,currency 1,Consumer credit,<NA>
305807,456250,6817237.000,-862.000000,0.0,1288.333333,-760.000000,0.000,0.000000,1.028820e+06,744013.365000,19422.79500,0.0,-60.333333,154567.965000,Active,currency 1,Consumer credit,X
305808,456253,5885018.500,-867.500000,0.0,280.500000,-794.000000,NaN,0.000000,9.900000e+05,448958.250000,0.00000,0.0,-253.250000,58369.500000,Active,currency 1,Consumer credit,C
305809,456254,6669849.000,-1104.000000,0.0,-859.000000,-859.000000,NaN,0.000000,4.500000e+04,0.000000,NaN,0.0,-401.000000,0.000000,Closed,currency 1,Consumer credit,C


In [ ]:
bureau_agg.isna().sum()

,0
SK_ID_CURR,0
SK_ID_BUREAU,0
DAYS_CREDIT,0
CREDIT_DAY_OVERDUE,0
DAYS_CREDIT_ENDDATE,2585
DAYS_ENDDATE_FACT,37656
AMT_CREDIT_MAX_OVERDUE,92840
CNT_CREDIT_PROLONG,0
AMT_CREDIT_SUM,2
AMT_CREDIT_SUM_DEBT,8372


In [ ]:
# Select object columns
object_cols_bureau = bureau_agg.select_dtypes(include='object')

# Create a DataFrame with feature names and unique counts
object_unique_counts_df = (
    object_cols_bureau.nunique()
    .reset_index()
    .rename(columns={'index': 'Feature', 0: 'UniqueValues'})
    .sort_values(by='UniqueValues')
)

print(object_unique_counts_df)


           Feature  UniqueValues
1  CREDIT_CURRENCY             3
0    CREDIT_ACTIVE             4
3      STATUS_MODE             7
2      CREDIT_TYPE            12


In [ ]:
# Select numeric columns
numeric_cols_bureau = bureau_agg.select_dtypes(include=['int64', 'float64']).columns

# Fill NaNs with column mean
bureau_agg[numeric_cols_bureau] = bureau_agg[numeric_cols_bureau].fillna(bureau_agg[numeric_cols_bureau].mean())
bureau_agg.drop('CREDIT_CURRENCY', axis=1, inplace=True)
bureau_agg['CREDIT_ACTIVE'] = bureau_agg['CREDIT_ACTIVE'].map({'Closed':0, 'Active':1, 'Sold':1, 'Bad debt':1})
bureau_agg.drop(['STATUS_MODE','AMT_ANNUITY'], axis=1, inplace=True)
bureau_agg['CREDIT_TYPE'] = bureau_agg['CREDIT_TYPE'].apply(
    lambda x: 1 if x == 'Consumer credit' else 2 if x == 'Credit card' else 3
)




In [ ]:
import pandas as pd

# Safe mode function
def safe_mode(s):
    m = s.mode()
    if len(m) == 0:
        return pd.NA  # or any default value
    else:
        return m[0]

# Separate numeric and object columns
numeric_cols = previous_application.select_dtypes(include='number').columns
object_cols = previous_application.select_dtypes(include='object').columns

# Aggregation dictionary
agg_dict = {col: 'mean' for col in numeric_cols}
agg_dict.update({col: safe_mode for col in object_cols})

# Group by SK_ID_CURR and aggregate
previous_agg = previous_application.groupby('SK_ID_CURR', as_index=False).agg(agg_dict)

# Ensure SK_ID_CURR is integer
previous_agg['SK_ID_CURR'] = previous_agg['SK_ID_CURR'].astype(int)

# View result
print(previous_agg.head())


     SK_ID_PREV  SK_ID_CURR  AMT_ANNUITY  AMT_APPLICATION  AMT_CREDIT  \
0  1.369693e+06      100001     3951.000         24835.50    23787.00   
1  1.038818e+06      100002     9251.775        179055.00   179055.00   
2  2.281150e+06      100003    56553.990        435436.50   484191.00   
3  1.564014e+06      100004     5357.250         24282.00    20106.00   
4  2.176837e+06      100005     4813.200         22308.75    20076.75   

   AMT_DOWN_PAYMENT  AMT_GOODS_PRICE  HOUR_APPR_PROCESS_START  \
0            2520.0          24835.5                13.000000   
1               0.0         179055.0                 9.000000   
2            3442.5         435436.5                14.666667   
3            4860.0          24282.0                 5.000000   
4            4464.0          44617.5                10.500000   

   NFLAG_LAST_APPL_IN_DAY  RATE_DOWN_PAYMENT  ...  CODE_REJECT_REASON  \
0                     1.0           0.104326  ...                 XAP   
1                     1.

In [ ]:
previous_agg.drop(columns=['RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED'], inplace=True)

In [ ]:
object_cols = previous_agg.select_dtypes(include='object').columns

# Show nunique values per object column
for col in object_cols:
    print(f"{col}: {previous_agg[col].nunique()}")

NAME_CONTRACT_TYPE: 4
WEEKDAY_APPR_PROCESS_START: 7
FLAG_LAST_APPL_PER_CONTRACT: 2
NAME_CASH_LOAN_PURPOSE: 25
NAME_CONTRACT_STATUS: 4
NAME_PAYMENT_TYPE: 4
CODE_REJECT_REASON: 9
NAME_TYPE_SUITE: 7
NAME_CLIENT_TYPE: 4
NAME_GOODS_CATEGORY: 26
NAME_PORTFOLIO: 5
NAME_PRODUCT_TYPE: 3
CHANNEL_TYPE: 8
NAME_SELLER_INDUSTRY: 11
NAME_YIELD_GROUP: 5
PRODUCT_COMBINATION: 17


In [ ]:
previous_agg['NAME_CONTRACT_TYPE'] = previous_agg['NAME_CONTRACT_TYPE'].map({'Consumer loans':0,
                                                                             'Cash loans':1, 'Revolving loans':2, 'XNA':2})
previous_agg.drop('WEEKDAY_APPR_PROCESS_START', axis=1, inplace=True)
previous_agg['NAME_CASH_LOAN_PURPOSE'] = previous_agg['NAME_CASH_LOAN_PURPOSE'].apply(lambda x: 1 if x=='XAP' else 2 if x=='XNA'
                                                                                      else 3)
previous_agg['NAME_CONTRACT_STATUS'] = previous_agg['NAME_CONTRACT_STATUS'].map({'Approved':1, 'Canceled':0, 'Refused':0, 'Unused offer':1})

previous_agg['NAME_PAYMENT_TYPE'] = previous_agg['NAME_PAYMENT_TYPE'].map({'Cash through the bank':1, 'Non-cash from your account':1,
                                                                           'Cashless from the account of the employer':1,
                                                                           'XNA':0})

previous_agg['CODE_REJECT_REASON'] = previous_agg['CODE_REJECT_REASON'].apply(lambda x: 1 if x=='XAP' else 0)

previous_agg.drop('NAME_TYPE_SUITE', axis=1, inplace=True)
previous_agg['NAME_CLIENT_TYPE'] = previous_agg['NAME_CLIENT_TYPE'].map({'Repeater':0, 'Refreshed':0,
                                                                         'New':1, 'XNA':1})
previous_agg.drop('NAME_GOODS_CATEGORY', axis=1, inplace=True)

previous_agg['NAME_PORTFOLIO'] = previous_agg['NAME_PORTFOLIO'].apply(lambda x: 1 if x=='POS' else 0 if x=='XNA'
                                                                      else 2)
previous_agg.drop('NAME_PRODUCT_TYPE', axis=1, inplace=True)
previous_agg.drop('NAME_SELLER_INDUSTRY', axis=1, inplace=True)
previous_agg.drop('NAME_YIELD_GROUP', axis=1, inplace=True)
previous_agg.drop('PRODUCT_COMBINATION', axis=1, inplace=True)


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Separate numeric and object columns
numeric_cols = previous_agg.select_dtypes(include='number').columns
object_cols = previous_agg.select_dtypes(include='object').columns

# 1️⃣ Fill missing numeric values with mean
previous_agg[numeric_cols] = previous_agg[numeric_cols].fillna(previous_agg[numeric_cols].mean())

# 2️⃣ Label encode object columns
for col in object_cols:
    le = LabelEncoder()
    previous_agg[col] = le.fit_transform(previous_agg[col].astype(str))  # convert to str to avoid errors

# Verify
print(previous_agg.head())


     SK_ID_PREV  SK_ID_CURR  AMT_ANNUITY  AMT_APPLICATION  AMT_CREDIT  \
0  1.369693e+06      100001     3951.000         24835.50    23787.00   
1  1.038818e+06      100002     9251.775        179055.00   179055.00   
2  2.281150e+06      100003    56553.990        435436.50   484191.00   
3  1.564014e+06      100004     5357.250         24282.00    20106.00   
4  2.176837e+06      100005     4813.200         22308.75    20076.75   

   AMT_DOWN_PAYMENT  AMT_GOODS_PRICE  HOUR_APPR_PROCESS_START  \
0            2520.0          24835.5                13.000000   
1               0.0         179055.0                 9.000000   
2            3442.5         435436.5                14.666667   
3            4860.0          24282.0                 5.000000   
4            4464.0          44617.5                10.500000   

   NFLAG_LAST_APPL_IN_DAY  RATE_DOWN_PAYMENT  ...  NFLAG_INSURED_ON_APPROVAL  \
0                     1.0           0.104326  ...                   0.000000   
1         

In [ ]:
previous_agg.drop('SK_ID_PREV', axis=1, inplace=True)
previous_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338857 entries, 0 to 338856
Data columns (total 27 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_CURR                   338857 non-null  int64  
 1   AMT_ANNUITY                  338857 non-null  float64
 2   AMT_APPLICATION              338857 non-null  float64
 3   AMT_CREDIT                   338857 non-null  float64
 4   AMT_DOWN_PAYMENT             338857 non-null  float64
 5   AMT_GOODS_PRICE              338857 non-null  float64
 6   HOUR_APPR_PROCESS_START      338857 non-null  float64
 7   NFLAG_LAST_APPL_IN_DAY       338857 non-null  float64
 8   RATE_DOWN_PAYMENT            338857 non-null  float64
 9   DAYS_DECISION                338857 non-null  float64
 10  SELLERPLACE_AREA             338857 non-null  float64
 11  CNT_PAYMENT                  338857 non-null  float64
 12  DAYS_FIRST_DRAWING           338857 non-null  float64
 13 

In [ ]:
pos_cash_balance.drop('SK_ID_PREV', axis=1, inplace=True)
install_payments.drop('SK_ID_PREV', axis=1, inplace=True)


In [ ]:
pos_cash_balance['NAME_CONTRACT_STATUS'] = pos_cash_balance['NAME_CONTRACT_STATUS'].apply(lambda x: 1 if x=='Active'
else 2 if x=='Completed' else 3)
# Aggregate by SK_ID_CURR taking mean for all columns
pos_cash_agg = pos_cash_balance.groupby('SK_ID_CURR', as_index=False).mean()

# Ensure SK_ID_CURR is integer
pos_cash_agg['SK_ID_CURR'] = pos_cash_agg['SK_ID_CURR'].astype(int)

print(pos_cash_agg.head())



   SK_ID_CURR  MONTHS_BALANCE  CNT_INSTALMENT  CNT_INSTALMENT_FUTURE  \
0      100001      -72.555556        4.000000               1.444444   
1      100002      -10.000000       24.000000              15.000000   
2      100003      -43.785714       10.107143               5.785714   
3      100004      -25.500000        3.750000               2.250000   
4      100005      -20.000000       11.700000               7.200000   

   NAME_CONTRACT_STATUS    SK_DPD  SK_DPD_DEF  
0              1.222222  0.777778    0.777778  
1              1.000000  0.000000    0.000000  
2              1.071429  0.000000    0.000000  
3              1.250000  0.000000    0.000000  
4              1.272727  0.000000    0.000000  


In [ ]:
# Aggregate by SK_ID_CURR taking mean for all columns
installments_agg = install_payments.groupby('SK_ID_CURR', as_index=False).mean()

# Ensure SK_ID_CURR is integer
installments_agg['SK_ID_CURR'] = installments_agg['SK_ID_CURR'].astype(int)

print(installments_agg.head())


   SK_ID_CURR  NUM_INSTALMENT_VERSION  NUM_INSTALMENT_NUMBER  DAYS_INSTALMENT  \
0      100001                1.142857               2.714286     -2187.714286   
1      100002                1.052632              10.000000      -295.000000   
2      100003                1.040000               5.080000     -1378.160000   
3      100004                1.333333               2.000000      -754.000000   
4      100005                1.111111               5.000000      -586.000000   

   DAYS_ENTRY_PAYMENT  AMT_INSTALMENT   AMT_PAYMENT  
0        -2195.000000     5885.132143   5885.132143  
1         -315.421053    11559.247105  11559.247105  
2        -1385.320000    64754.586000  64754.586000  
3         -761.666667     7096.155000   7096.155000  
4         -609.555556     6240.205000   6240.205000  


In [ ]:
pos_cash_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337252 entries, 0 to 337251
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   SK_ID_CURR             337252 non-null  int64  
 1   MONTHS_BALANCE         337252 non-null  float64
 2   CNT_INSTALMENT         337224 non-null  float64
 3   CNT_INSTALMENT_FUTURE  337224 non-null  float64
 4   NAME_CONTRACT_STATUS   337252 non-null  float64
 5   SK_DPD                 337252 non-null  float64
 6   SK_DPD_DEF             337252 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 18.0 MB


In [ ]:
installments_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339587 entries, 0 to 339586
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   SK_ID_CURR              339587 non-null  int64  
 1   NUM_INSTALMENT_VERSION  339587 non-null  float64
 2   NUM_INSTALMENT_NUMBER   339587 non-null  float64
 3   DAYS_INSTALMENT         339587 non-null  float64
 4   DAYS_ENTRY_PAYMENT      339578 non-null  float64
 5   AMT_INSTALMENT          339587 non-null  float64
 6   AMT_PAYMENT             339578 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 18.1 MB


In [ ]:
# Fill NaNs in numeric columns with mean for pos_cash_agg
pos_cash_agg = pos_cash_agg.fillna(pos_cash_agg.mean())

# Fill NaNs in numeric columns with mean for installments_agg
installments_agg = installments_agg.fillna(installments_agg.mean())

# Verify
print(pos_cash_agg.isna().sum().sum())  # Should be 0
print(installments_agg.isna().sum().sum())  # Should be 0


0
0


# Merge of datasets

In [ ]:
datasets_list = [X, bureau_agg, previous_agg, pos_cash_agg, installments_agg]

for dataset in datasets_list:
    print(dataset.isna().sum().sum())

0
0
0
0
0


In [ ]:
# List of datasets to merge (excluding X)
datasets_list = [bureau_agg, previous_agg, pos_cash_agg, installments_agg]

# Merge each dataset into X on SK_ID_CURR
for df in datasets_list:
    X = X.merge(df, on='SK_ID_CURR', how='left')

# Verify
print(X.shape)
print(X.head())


(307511, 141)
   SK_ID_CURR  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT_x  AMT_ANNUITY_x  \
0      100002             0          202500.0      406597.5        24700.5   
1      100003             0          270000.0     1293502.5        35698.5   
2      100004             0           67500.0      135000.0         6750.0   
3      100006             0          135000.0      312682.5        29686.5   
4      100007             0          121500.0      513000.0        21865.5   

   AMT_GOODS_PRICE_x  NAME_TYPE_SUITE  NAME_INCOME_TYPE  NAME_EDUCATION_TYPE  \
0           351000.0                0                 1                    1   
1          1129500.0                1                 2                    2   
2           135000.0                0                 1                    1   
3           297000.0                0                 1                    1   
4           513000.0                0                 1                    1   

   NAME_FAMILY_STATUS  ...  CNT_INST

In [ ]:
# Fill all NaNs in X with column mean
X = X.fillna(X.mean())

# Verify no missing values remain
print(X.isna().sum().sum())  # Should print 0


0


# Creating Pipelines

In [ ]:
X.shape

(307511, 141)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

class CustomPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, one_hot, cols_over_50_nan, merge_datasets=None):
        self.one_hot = one_hot
        self.cols_over_50_nan = cols_over_50_nan
        self.merge_datasets = merge_datasets if merge_datasets else []
        self.encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
        self.encoded_cols = None

    def fit(self, X, y=None):
        X = X.copy()

        # Fix CODE_GENDER and EMERGENCYSTATE_MODE BEFORE encoding
        if "CODE_GENDER" in X.columns:
            X["CODE_GENDER"] = X["CODE_GENDER"].replace("XNA", "F")
        if "EMERGENCYSTATE_MODE" in X.columns:
            X["EMERGENCYSTATE_MODE"] = X["EMERGENCYSTATE_MODE"].fillna("No")

        # Fit encoder on cleaned categorical cols
        self.encoder.fit(X[self.one_hot])
        self.encoded_cols = self.encoder.get_feature_names_out(self.one_hot)

        return self

    def transform(self, X):
        X = X.copy()

        # 1. Drop cols with >50% NaN
        X.drop(self.cols_over_50_nan, axis=1, inplace=True, errors="ignore")

        # 2. Fill NaN as per manual strategy
        fill_mean = [
            'AMT_ANNUITY','AMT_GOODS_PRICE','CNT_FAM_MEMBERS','EXT_SOURCE_2','EXT_SOURCE_3',
            'YEARS_BEGINEXPLUATATION_AVG','FLOORSMAX_AVG',
            'YEARS_BEGINEXPLUATATION_MODE','FLOORSMAX_MODE',
            'YEARS_BEGINEXPLUATATION_MEDI','FLOORSMAX_MEDI','TOTALAREA_MODE'
        ]
        for col in fill_mean:
            if col in X.columns:
                X[col] = X[col].fillna(X[col].mean())

        fill_mode = [
            'OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE',
            'OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE',
            'DAYS_LAST_PHONE_CHANGE',
            'AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY',
            'AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON',
            'AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR'
        ]
        for col in fill_mode:
            if col in X.columns:
                X[col] = X[col].fillna(X[col].mode()[0])

        if "NAME_TYPE_SUITE" in X.columns:
            X["NAME_TYPE_SUITE"] = X["NAME_TYPE_SUITE"].fillna("Unidentified")
        if "OCCUPATION_TYPE" in X.columns:
            X["OCCUPATION_TYPE"] = X["OCCUPATION_TYPE"].fillna("Unemployed")

        # 3. Manual categorical mappings
        if "CODE_GENDER" in X.columns:
            X["CODE_GENDER"] = X["CODE_GENDER"].replace("XNA", "F")
        if "EMERGENCYSTATE_MODE" in X.columns:
            X["EMERGENCYSTATE_MODE"] = X["EMERGENCYSTATE_MODE"].fillna("No")

        if "NAME_EDUCATION_TYPE" in X.columns:
            X["NAME_EDUCATION_TYPE"] = X["NAME_EDUCATION_TYPE"].map({
                'Secondary / secondary special': 1,'Lower secondary': 1,
                'Higher education': 2,'Incomplete higher': 2,'Academic degree': 2
            })
        if "NAME_FAMILY_STATUS" in X.columns:
            X["NAME_FAMILY_STATUS"] = X["NAME_FAMILY_STATUS"].map({
                'Married':1,'Civil marriage':1,
                'Single / not married':2,'Separated':2,'Widow':2,'Unknown':2
            })
        if "NAME_INCOME_TYPE" in X.columns:
            X["NAME_INCOME_TYPE"] = X["NAME_INCOME_TYPE"].map({
                'Working':1,'Commercial associate':1,'Pensioner':2,'State servant':2,
                'Unemployed':3,'Student':3,'Maternity leave':3,'Businessman':0
            })
        if "NAME_TYPE_SUITE" in X.columns:
            X["NAME_TYPE_SUITE"] = X["NAME_TYPE_SUITE"].map({
                'Unaccompanied':0,'Family':1,'Spouse, partner':2,'Children':1,
                'Other_B':3,'Other_A':3,'Unidentified':3,'Group of people':3
            })
        if "OCCUPATION_TYPE" in X.columns:
            X["OCCUPATION_TYPE"] = X["OCCUPATION_TYPE"].map({
                'Unemployed':0,
                'Low-skill Laborers':1,'Cleaning staff':1,'Cooking staff':1,'Waiters/barmen staff':1,'Private service staff':1,
                'Laborers':2,'Sales staff':2,'Security staff':2,'Drivers':2,
                'Core staff':3,'Medicine staff':3,'Secretaries':3,
                'Accountants':4,'HR staff':4,'Realty agents':4,
                'Managers':5,'High skill tech staff':5,'IT staff':5
            })
        if "ORGANIZATION_TYPE" in X.columns:
            X["ORGANIZATION_TYPE"] = X["ORGANIZATION_TYPE"].map({
                'XNA':0,'Other':0,'Religion':0,'Kindergarten':0,'School':0,'Culture':0,'Cleaning':0,
                'Services':1,'Restaurant':1,'Hotel':1,
                'Trade: type 1':1,'Trade: type 2':1,'Trade: type 3':1,'Trade: type 4':1,
                'Trade: type 5':1,'Trade: type 6':1,'Trade: type 7':1,
                'Agriculture':2,'Industry: type 1':2,'Industry: type 2':2,'Industry: type 3':2,
                'Industry: type 4':2,'Industry: type 5':2,'Industry: type 6':2,'Industry: type 7':2,
                'Industry: type 8':2,'Industry: type 9':2,'Industry: type 10':2,'Industry: type 11':2,
                'Industry: type 12':2,'Industry: type 13':2,'Construction':2,'Housing':2,
                'Transport: type 1':3,'Transport: type 2':3,'Transport: type 3':3,'Transport: type 4':3,
                'Electricity':3,'Security':3,'Postal':3,
                'Government':4,'Military':4,'Police':4,'Security Ministries':4,'Emergency':4,'University':4,
                'Self-employed':5,'Business Entity Type 1':5,'Business Entity Type 2':5,'Business Entity Type 3':5,'Realtor':5,
                'Medicine':6,'Legal Services':6,'Advertising':6,'Mobile':6,'Telecom':6,'Insurance':6,'Bank':6
            })

        # 4. One-hot encoding (AFTER fixing XNA + EMERGENCYSTATE_MODE)
        encoded_array = self.encoder.transform(X[self.one_hot])
        encoded_df = pd.DataFrame(encoded_array, columns=self.encoded_cols, index=X.index)
        X = pd.concat([X.drop(columns=self.one_hot), encoded_df], axis=1)

        # 5. Merge external datasets
        for df in self.merge_datasets:
            X = X.merge(df, on="SK_ID_CURR", how="left")

        # 6. Final fillna with mean for numeric
        X = X.fillna(X.mean(numeric_only=True))

        return X


# ----------------- USE THE PIPELINE -----------------



pipeline = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(one_hot=one_hot,
                                        cols_over_50_nan=cols_over_50_nan,
                                        merge_datasets=datasets_list))
])




In [ ]:
D = train.drop(['TARGET'], axis=1)

In [ ]:
pipeline.fit(D)
X_processed = pipeline.transform(D)

# Quick checks
print("Shape of processed X:", X_processed.shape)
print("Any missing values left?", X_processed.isna().sum().sum())
print("Columns sample:", X_processed.columns[:20])


Shape of processed X: (307511, 141)
Any missing values left? 0
Columns sample: Index(['SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT_x',
       'AMT_ANNUITY_x', 'AMT_GOODS_PRICE_x', 'NAME_TYPE_SUITE',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_MOBIL', 'FLAG_EMP_PHONE',
       'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE'],
      dtype='object')


/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [ ]:
# Columns in X but not in X_processed
cols_removed = set(X.columns) - set(X_processed.columns)

# Columns in X_processed but not in X
cols_added = set(X_processed.columns) - set(X.columns)

# Columns in both
cols_unchanged = set(X.columns).intersection(set(X_processed.columns))

print("Removed columns:", cols_removed)
print("Added columns:", cols_added)
print("Unchanged columns:", cols_unchanged)


Removed columns: set()
Added columns: set()
Unchanged columns: {'FLAG_DOCUMENT_7', 'AMT_CREDIT_SUM', 'DEF_60_CNT_SOCIAL_CIRCLE', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_y', 'DAYS_TERMINATION', 'CODE_REJECT_REASON', 'NAME_HOUSING_TYPE_With parents', 'WEEKDAY_APPR_PROCESS_START_THURSDAY', 'NUM_INSTALMENT_VERSION', 'AMT_CREDIT_SUM_OVERDUE', 'WEEKDAY_APPR_PROCESS_START_WEDNESDAY', 'YEARS_BEGINEXPLUATATION_AVG', 'FLAG_OWN_CAR_Y', 'FLAG_DOCUMENT_8', 'AMT_INCOME_TOTAL', 'DAYS_FIRST_DUE', 'NUM_INSTALMENT_NUMBER', 'REGION_RATING_CLIENT_W_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_EMP_PHONE', 'FLAG_DOCUMENT_13', 'REGION_POPULATION_RELATIVE', 'DAYS_DECISION', 'CNT_FAM_MEMBERS', 'FLAG_EMAIL', 'ORGANIZATION_TYPE', 'WEEKDAY_APPR_PROCESS_START_MONDAY', 'NAME_PORTFOLIO', 'CNT_PAYMENT', 'FLAG_DOCUMENT_21', 'FLAG_DOCUMENT_11', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_CONTRACT_TYPE_Revolving loans', 'FLAG_MOBIL', 'FLAG_OWN_REALTY_Y', 'DAYS_BIRTH', 'DAYS_LAST_DUE_1ST_VERSION', 'EXT_SOURCE_2', 'REG_REGION

In [ ]:
all_numeric = X_processed.select_dtypes(include=['number']).shape[1] == X_processed.shape[1]
print("All numeric:", all_numeric)


All numeric: True


In [ ]:
print("Any NaN:", X_processed.isna().any().any())
print("Any Inf:", np.isinf(X_processed.to_numpy()).any())


Any NaN: False
Any Inf: False


# Baseline models

In [ ]:
X = train.drop('TARGET', axis=1)
y = train['TARGET']

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

log_pipeline = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(one_hot=one_hot,
                                        cols_over_50_nan=cols_over_50_nan,
                                        merge_datasets=datasets_list)),
    ("scaler", StandardScaler()),   # scales float features
    ("pca", PCA(n_components=0.99, random_state=42)),  # keeps 99% variance
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_pipeline = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(one_hot=one_hot,
                                        cols_over_50_nan=cols_over_50_nan,
                                        merge_datasets=datasets_list)),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", RandomForestClassifier(n_estimators=100, random_state=42))
])


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb_pipeline = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(one_hot=one_hot,
                                        cols_over_50_nan=cols_over_50_nan,
                                        merge_datasets=datasets_list)),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", GradientBoostingClassifier(random_state=42))
])


In [ ]:
from sklearn.svm import SVC

svc_pipeline = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(one_hot=one_hot,
                                        cols_over_50_nan=cols_over_50_nan,
                                        merge_datasets=datasets_list)),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", SVC(probability=True, random_state=42))
])


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_pipeline = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(one_hot=one_hot,
                                        cols_over_50_nan=cols_over_50_nan,
                                        merge_datasets=datasets_list)),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", KNeighborsClassifier())
])


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_pipeline = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(one_hot=one_hot,
                                        cols_over_50_nan=cols_over_50_nan,
                                        merge_datasets=datasets_list)),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", GaussianNB())
])


In [ ]:
from lightgbm import LGBMClassifier

lgbm_pipeline = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(one_hot=one_hot,
                                        cols_over_50_nan=cols_over_50_nan,
                                        merge_datasets=datasets_list)),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", LGBMClassifier(random_state=42))
])


In [ ]:
from xgboost import XGBClassifier

xgb_pipeline = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(one_hot=one_hot,
                                        cols_over_50_nan=cols_over_50_nan,
                                        merge_datasets=datasets_list)),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
])


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_pipeline = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(one_hot=one_hot,
                                        cols_over_50_nan=cols_over_50_nan,
                                        merge_datasets=datasets_list)),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", AdaBoostClassifier(random_state=42))
])


In [ ]:
pip install catboost


In [ ]:
from catboost import CatBoostClassifier

cat_pipeline = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(one_hot=one_hot,
                                        cols_over_50_nan=cols_over_50_nan,
                                        merge_datasets=datasets_list)),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", CatBoostClassifier(verbose=0, random_state=42))
])


# Testing Models

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np

# Define stratified CV
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Evaluate with cross_val_score (ROC-AUC is good for classification)
scores = cross_val_score(log_pipeline, X, y, cv=skf, scoring="roc_auc")

print("ROC-AUC scores:", scores)
print("Mean ROC-AUC:", np.mean(scores))

# ROC-AUC scores: [0.74753645 0.74947381 0.74817977]
# Mean ROC-AUC: 0.7483966768269751

ROC-AUC scores: [0.74753645 0.74947381 0.74817977]
Mean ROC-AUC: 0.7483966768269751


In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# ROC-AUC scoring
roc_auc_scores = cross_val_score(
    rf_pipeline,
    X,
    y,
    cv=cv,
    scoring="roc_auc",
    n_jobs=-1
)

print("ROC-AUC scores:", roc_auc_scores)
print("Mean ROC-AUC:", np.mean(roc_auc_scores))

# ROC-AUC scores: [0.6704201  0.66548507 0.66156737]
# Mean ROC-AUC: 0.6658241813699064

ROC-AUC scores: [0.6704201  0.66548507 0.66156737]
Mean ROC-AUC: 0.6658241813699064


In [ ]:
# Stratified 3-fold CV
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# ROC-AUC scoring
roc_auc_scores = cross_val_score(
    gb_pipeline,
    X,
    y,
    cv=cv,
    scoring="roc_auc",
    n_jobs=-1
)

print("ROC-AUC scores:", roc_auc_scores)
print("Mean ROC-AUC:", np.mean(roc_auc_scores))

# ROC-AUC scores: [0.73105384 0.73206156 0.73087152]
# Mean ROC-AUC: 0.7313289711429025

ROC-AUC scores: [0.73105384 0.73206156 0.73087152]
Mean ROC-AUC: 0.7313289711429025


In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

roc_auc_scores = cross_val_score(
    knn_pipeline, X, y, cv=cv, scoring="roc_auc", n_jobs=-1
)

print("KNN ROC-AUC scores:", roc_auc_scores)
print("Mean ROC-AUC:", np.mean(roc_auc_scores))

KNN ROC-AUC scores: [0.57717754 0.57681129 0.57100998]
Mean ROC-AUC: 0.5749996027472778


In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

roc_auc_scores = cross_val_score(
    nb_pipeline, X, y, cv=cv, scoring="roc_auc", n_jobs=-1
)

print("Naive Bayes ROC-AUC scores:", roc_auc_scores)
print("Mean ROC-AUC:", np.mean(roc_auc_scores))

Naive Bayes ROC-AUC scores: [0.6654625  0.64818286 0.65428832]
Mean ROC-AUC: 0.6559778933857418


In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

roc_auc_scores = cross_val_score(
    lgbm_pipeline, X, y, cv=cv, scoring="roc_auc", n_jobs=-1
)

print("LightGBM ROC-AUC scores:", roc_auc_scores)
print("Mean ROC-AUC:", np.mean(roc_auc_scores))

# LightGBM ROC-AUC scores: [0.73657282 0.74003513 0.73908695]
# Mean ROC-AUC: 0.7385649656135019


/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


LightGBM ROC-AUC scores: [0.73657282 0.74003513 0.73908695]
Mean ROC-AUC: 0.7385649656135019


In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

roc_auc_scores = cross_val_score(
    xgb_pipeline, X, y, cv=cv, scoring="roc_auc", n_jobs=-1
)

print("XGBoost ROC-AUC scores:", roc_auc_scores)
print("Mean ROC-AUC:", np.mean(roc_auc_scores))

XGBoost ROC-AUC scores: [0.71828999 0.72283743 0.72006924]
Mean ROC-AUC: 0.7203988842946064


In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

roc_auc_scores = cross_val_score(
    ada_pipeline, X, y, cv=cv, scoring="roc_auc", n_jobs=-1
)

print("AdaBoost ROC-AUC scores:", roc_auc_scores)
print("Mean ROC-AUC:", np.mean(roc_auc_scores))

AdaBoost ROC-AUC scores: [0.71020027 0.71251092 0.70875433]
Mean ROC-AUC: 0.7104885072379304


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

roc_auc_scores = cross_val_score(
    cat_pipeline, X, y, cv=cv, scoring="roc_auc", n_jobs=-1
)

print("CatBoost ROC-AUC scores:", roc_auc_scores)
print("Mean ROC-AUC:", np.mean(roc_auc_scores))

# CatBoost ROC-AUC scores: [0.74040066 0.74398393 0.7422809 ]
# Mean ROC-AUC: 0.742221828948819

CatBoost ROC-AUC scores: [0.74040066 0.74398393 0.7422809 ]
Mean ROC-AUC: 0.742221828948819


# Hyperparameter tuning

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Example objective function
def objective(trial):
    # Suggest hyperparameters for CatBoost
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "border_count": trial.suggest_int("border_count", 32, 128),
        "verbose": 0,
        "random_state": 42
    }

    # Create pipeline with CatBoostClassifier using suggested params
    cat_tune = Pipeline([
        ("preprocessor", CustomPreprocessor(
            one_hot=one_hot,
            cols_over_50_nan=cols_over_50_nan,
            merge_datasets=datasets_list
        )),
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=0.95, random_state=42)),
        ("classifier", CatBoostClassifier(**params))
    ])

    # Evaluate using cross-validation
    scores = cross_val_score(cat_tune, X, y, cv=3, scoring="roc_auc", n_jobs=-1)

    return scores.mean()  # Optuna maximizes this by default

# Create study
study = optuna.create_study(direction="maximize")  # maximize ROC-AUC
study.optimize(objective, n_trials=20)  # number of trials

print("Best ROC-AUC:", study.best_value)
print("Best hyperparameters:", study.best_params)


# {'iterations': 672, 'depth': 4, 'learning_rate': 0.0600444514342209, 'l2_leaf_reg': 5.087392359193165, 'border_count': 44}. Best is trial 0 with value: 0.7436044779844919.

[I 2025-09-14 10:20:38,382] A new study created in memory with name: no-name-a778c992-b8de-4f7c-a0e2-8184512d62a0
[I 2025-09-14 10:25:13,490] Trial 0 finished with value: 0.7436044779844919 and parameters: {'iterations': 672, 'depth': 4, 'learning_rate': 0.0600444514342209, 'l2_leaf_reg': 5.087392359193165, 'border_count': 44}. Best is trial 0 with value: 0.7436044779844919.
[I 2025-09-14 10:31:48,430] Trial 1 finished with value: 0.7325694199990354 and parameters: {'iterations': 982, 'depth': 5, 'learning_rate': 0.17992618644326608, 'l2_leaf_reg': 8.722632451998155, 'border_count': 32}. Best is trial 0 with value: 0.7436044779844919.
[I 2025-09-14 10:34:00,940] Trial 2 finished with value: 0.7316621342979527 and parameters: {'iterations': 131, 'depth': 7, 'learning_rate': 0.049223162971613464, 'l2_leaf_reg': 7.994899156885492, 'border_count': 102}. Best is trial 0 with value: 0.7436044779844919.
[I 2025-09-14 10:42:46,309] Trial 3 finished with value: 0.7262228689604092 and parameters

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier

# Your tuned parameters
best_params = {
    'iterations': 672,
    'depth': 4,
    'learning_rate': 0.0600444514342209,
    'l2_leaf_reg': 5.087392359193165,
    'border_count': 44
}

# Define tuned pipeline
cat_pipeline_tuned = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(
        one_hot=one_hot,
        cols_over_50_nan=cols_over_50_nan,
        merge_datasets=datasets_list
    )),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", CatBoostClassifier(
        verbose=0,
        random_state=42,
        **best_params
    ))
])

# Cross-validate with ROC AUC
scores = cross_val_score(
    cat_pipeline_tuned, X, y,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1
)

print("ROC AUC scores:", scores)
print("Mean ROC AUC:", scores.mean())


# ROC AUC scores: [0.74751152 0.74694089 0.74255921 0.74828448 0.74889171]
# Mean ROC AUC: 0.7468375632133442

ROC AUC scores: [0.74751152 0.74694089 0.74255921 0.74828448 0.74889171]
Mean ROC AUC: 0.7468375632133442


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from xgboost import XGBClassifier

# Define objective function for Optuna
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10, log=True),
    }

    xgb_pipeline_tune = Pipeline(steps=[
        ("preprocessor", CustomPreprocessor(
            one_hot=one_hot,
            cols_over_50_nan=cols_over_50_nan,
            merge_datasets=datasets_list
        )),
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=0.95, random_state=42)),
        ("classifier", XGBClassifier(
            use_label_encoder=False,
            eval_metric="logloss",
            random_state=42,
            **params
        ))
    ])

    # Cross-validation with ROC AUC
    score = cross_val_score(
        xgb_pipeline_tune, X, y,
        cv=3,
        scoring="roc_auc",
        n_jobs=-1
    ).mean()

    return score

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  # increase trials for better tuning

print("Best params:", study.best_params)
print("Best ROC AUC:", study.best_value)


# {'n_estimators': 624, 'max_depth': 5, 'learning_rate': 0.04285419773446299, 'subsample': 0.6829863255306177, 'colsample_bytree': 0.5091030349379196, 'gamma': 4.932111410168664, 'reg_alpha': 0.001281424078178488, 'reg_lambda': 0.0016101299020309864}. Best is trial 10 with value: 0.743781246190931.

[I 2025-09-14 11:58:29,900] A new study created in memory with name: no-name-b1eaecbe-e55f-4ea6-8fe6-a441a7ecdab4
[I 2025-09-14 12:02:01,369] Trial 0 finished with value: 0.7280200787728859 and parameters: {'n_estimators': 831, 'max_depth': 3, 'learning_rate': 0.009870398409968683, 'subsample': 0.9997060220641589, 'colsample_bytree': 0.6727139169967586, 'gamma': 1.8310674060625876, 'reg_alpha': 0.010580672719037478, 'reg_lambda': 0.2785820601640887}. Best is trial 0 with value: 0.7280200787728859.
[I 2025-09-14 12:16:14,685] Trial 1 finished with value: 0.7410529194479426 and parameters: {'n_estimators': 974, 'max_depth': 8, 'learning_rate': 0.005077321431469178, 'subsample': 0.7629126902838601, 'colsample_bytree': 0.8318223945578587, 'gamma': 0.35731597545783256, 'reg_alpha': 1.5620820316149617, 'reg_lambda': 0.029582672762297924}. Best is trial 1 with value: 0.7410529194479426.
[I 2025-09-14 12:21:42,964] Trial 2 finished with value: 0.7374028476002777 and parameters: {'n_estimators'

In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

xgb_pipeline_tuned = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(
        one_hot=one_hot,
        cols_over_50_nan=cols_over_50_nan,
        merge_datasets=datasets_list
    )),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", XGBClassifier(
        n_estimators=624,
        max_depth=5,
        learning_rate=0.04285419773446299,
        subsample=0.6829863255306177,
        colsample_bytree=0.5091030349379196,
        gamma=4.932111410168664,
        reg_alpha=0.001281424078178488,
        reg_lambda=0.0016101299020309864,
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42
    ))
])


In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Stratified 3-fold CV
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Cross-validation scores with ROC-AUC
scores = cross_val_score(
    xgb_pipeline_tuned,
    X,
    y,
    cv=cv,
    scoring="roc_auc",
    n_jobs=-1
)

print("ROC-AUC scores:", scores)
print("Mean ROC-AUC:", scores.mean())

# ROC-AUC scores: [0.74379538 0.74701401 0.74687096]
# Mean ROC-AUC: 0.7458934505508137

ROC-AUC scores: [0.74379538 0.74701401 0.74687096]
Mean ROC-AUC: 0.7458934505508137


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score, StratifiedKFold
from lightgbm import LGBMClassifier

# Adjusted pipeline
lgbm_pipeline_tuned = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(
        one_hot=one_hot,
        cols_over_50_nan=cols_over_50_nan,
        merge_datasets=datasets_list
    )),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.95, random_state=42)),  # lower PCA to save RAM
    ("classifier", LGBMClassifier(random_state=42, n_jobs=1))  # force single-threaded LGBM
])

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 800),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "num_leaves": trial.suggest_int("num_leaves", 15, 100),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
    }

    # Clone pipeline with params
    pipeline = lgbm_pipeline_tuned.set_params(
        classifier=LGBMClassifier(
            random_state=42,
            n_jobs=1,  # avoid parallel RAM overload
            **params
        )
    )

    # 3-fold CV instead of 5-fold
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(
        pipeline, X, y,
        cv=cv,
        scoring="roc_auc",
        n_jobs=1  # single-thread CV
    )

    return scores.mean()

# Run Optuna with fewer trials
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, n_jobs=1)  # reduce trials to save RAM

print("Best ROC AUC:", study.best_value)
print("Best params:", study.best_params)

# [I 2025-09-14 17:13:30,970] Trial 13 finished with value: 0.7430440707363336 and parameters: {'n_estimators': 792, 'learning_rate': 0.01859467742347784, 'max_depth': 5, 'num_leaves': 39, 'subsample': 0.7080921347459717, 'colsample_bytree': 0.8105402487129176}. Best is trial 13 with value: 0.7430440707363336.

[I 2025-09-14 16:21:39,935] A new study created in memory with name: no-name-ea66c945-ba55-412c-b60f-5cc0365b09a2


[LightGBM] [Info] Number of positive: 16550, number of negative: 188457
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.267020 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25755
[LightGBM] [Info] Number of data points in the train set: 205007, number of used features: 101
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432484
[LightGBM] [Info] Start training from score -2.432484


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 16550, number of negative: 188457
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.322030 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25755
[LightGBM] [Info] Number of data points in the train set: 205007, number of used features: 101
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432484
[LightGBM] [Info] Start training from score -2.432484


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

[LightGBM] [Info] Number of positive: 16550, number of negative: 188457
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.252503 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25755
[LightGBM] [Info] Number of data points in the train set: 205007, number of used features: 101
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432484
[LightGBM] [Info] Start training from score -2.432484
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 16550, number of negative: 188457
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.433860 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25755
[LightGBM] [Info] Number of data points in the train set: 205007, number of used features: 101
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432484
[LightGBM] [Info] Start training from score -2.432484
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 16550, number of negative: 188458
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.260593 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 205008, number of used features: 100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432489
[LightGBM] [Info] Start training from score -2.432489
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from lightgbm import LGBMClassifier

# Best parameters from tuning
best_params = {
    'n_estimators': 792,
    'learning_rate': 0.01859467742347784,
    'max_depth': 5,
    'num_leaves': 39,
    'subsample': 0.7080921347459717,
    'colsample_bytree': 0.8105402487129176
}

# Define pipeline with tuned LGBM
lgbm_pipeline_n = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(
        one_hot=one_hot,
        cols_over_50_nan=cols_over_50_nan,
        merge_datasets=datasets_list
    )),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.99, random_state=42)),
    ("classifier", LGBMClassifier(
        random_state=42,
        **best_params
    ))
])

# Stratified KFold for class balance
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation ROC-AUC
scores = cross_val_score(
    lgbm_pipeline_n, X, y,
    cv=cv,
    scoring="roc_auc",
    n_jobs=1  # safe for RAM
)

print("ROC-AUC scores:", scores)
print("Mean ROC-AUC:", scores.mean())

# ROC-AUC scores: [0.7406089  0.75117933 0.74352262 0.74795644 0.74267389]
# Mean ROC-AUC: 0.745188236132371


[LightGBM] [Info] Number of positive: 19860, number of negative: 226148
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.624790 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29580
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 116
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432482
[LightGBM] [Info] Start training from score -2.432482
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 19860, number of negative: 226149
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.603538 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29580
[LightGBM] [Info] Number of data points in the train set: 246009, number of used features: 116
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432486
[LightGBM] [Info] Start training from score -2.432486
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 19860, number of negative: 226149
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.605355 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29580
[LightGBM] [Info] Number of data points in the train set: 246009, number of used features: 116
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432486
[LightGBM] [Info] Start training from score -2.432486
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

...

[LightGBM] [Info] Number of positive: 19860, number of negative: 226149
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.366666 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29580
[LightGBM] [Info] Number of data points in the train set: 246009, number of used features: 116
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432486
[LightGBM] [Info] Start training from score -2.432486
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 19860, number of negative: 226149
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.347948 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29325
[LightGBM] [Info] Number of data points in the train set: 246009, number of used features: 115
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432486
[LightGBM] [Info] Start training from score -2.432486
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


ROC-AUC scores: [0.7406089  0.75117933 0.74352262 0.74795644 0.74267389]
Mean ROC-AUC: 0.745188236132371


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression

log_pipeline_kbest = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(
        one_hot=one_hot,
        cols_over_50_nan=cols_over_50_nan,
        merge_datasets=datasets_list
    )),
    ("scaler", StandardScaler()),  # scale numeric features

    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])


In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

# 3-fold stratified CV
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

scores = cross_val_score(
    log_pipeline_kbest, X, y,
    cv=cv,
    scoring="roc_auc",
    n_jobs=1  # keep single-threaded to save RAM
)

print("ROC-AUC scores:", scores)
print("Mean ROC-AUC:", scores.mean())

# ROC-AUC scores: [0.75181267 0.75601002 0.75302119]
# Mean ROC-AUC: 0.7536146273339135

ROC-AUC scores: [0.75181267 0.75601002 0.75302119]
Mean ROC-AUC: 0.7536146273339135


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier

# Your tuned parameters
best_params = {
    'iterations': 672,
    'depth': 4,
    'learning_rate': 0.0600444514342209,
    'l2_leaf_reg': 5.087392359193165,
    'border_count': 44
}

# Define tuned pipeline
cat_pipeline_tuned = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(
        one_hot=one_hot,
        cols_over_50_nan=cols_over_50_nan,
        merge_datasets=datasets_list
    )),
    ("scaler", StandardScaler()),

    ("classifier", CatBoostClassifier(thread_count=1,
        verbose=0,
        random_state=42,
        **best_params
    ))
])



In [ ]:
# Cross-validate with ROC AUC
scores = cross_val_score(
    cat_pipeline_tuned, X, y,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1
)

print("ROC AUC scores:", scores)
print("Mean ROC AUC:", scores.mean())


# ROC AUC scores: [0.77205479 0.77108597 0.76537767 0.77163089 0.77104556]
# Mean ROC AUC: 0.770238974765775

In [ ]:
# Best parameters from tuning
best_params = {
    'n_estimators': 792,
    'learning_rate': 0.01859467742347784,
    'max_depth': 5,
    'num_leaves': 39,
    'subsample': 0.7080921347459717,
    'colsample_bytree': 0.8105402487129176
}

# Define pipeline with tuned LGBM
lgbm_pipeline_n = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(
        one_hot=one_hot,
        cols_over_50_nan=cols_over_50_nan,
        merge_datasets=datasets_list
    )),
    ("scaler", StandardScaler()),

    ("classifier", LGBMClassifier(n_jobs=1,
        random_state=42,
        **best_params
    ))
])

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

# 3-fold stratified CV
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Cross-validation
scores = cross_val_score(
    lgbm_pipeline_n,
    X, y,
    cv=cv,
    scoring="roc_auc",
    n_jobs=1   # single-threaded to save RAM
)

print("ROC-AUC scores:", scores)
print("Mean ROC-AUC:", scores.mean())
print("Std ROC-AUC:", scores.std())

# ROC-AUC scores: [0.76961633 0.77066064 0.76595566]
# Mean ROC-AUC: 0.7687442086574817
# Std ROC-AUC: 0.002017367573670068


[LightGBM] [Info] Number of positive: 16550, number of negative: 188457
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.364688 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13986
[LightGBM] [Info] Number of data points in the train set: 205007, number of used features: 136
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432484
[LightGBM] [Info] Start training from score -2.432484
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 16550, number of negative: 188457
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.156604 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13918
[LightGBM] [Info] Number of data points in the train set: 205007, number of used features: 136
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432484
[LightGBM] [Info] Start training from score -2.432484
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 16550, number of negative: 188458
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.172586 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13915
[LightGBM] [Info] Number of data points in the train set: 205008, number of used features: 137
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432489
[LightGBM] [Info] Start training from score -2.432489
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


ROC-AUC scores: [0.76961633 0.77066064 0.76595566]
Mean ROC-AUC: 0.7687442086574817
Std ROC-AUC: 0.002017367573670068


In [ ]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

xgb_pipeline_tuned = Pipeline(steps=[
    ("preprocessor", CustomPreprocessor(
        one_hot=one_hot,
        cols_over_50_nan=cols_over_50_nan,
        merge_datasets=datasets_list
    )),
    ("scaler", StandardScaler()),

    ("classifier", XGBClassifier(n_jobs=1,
        n_estimators=624,
        max_depth=5,
        learning_rate=0.04285419773446299,
        subsample=0.6829863255306177,
        colsample_bytree=0.5091030349379196,
        gamma=4.932111410168664,
        reg_alpha=0.001281424078178488,
        reg_lambda=0.0016101299020309864,
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42
    ))
])





In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
# Cross-validation
scores = cross_val_score(
    xgb_pipeline_tuned,
    X, y,
    cv=cv,
    scoring="roc_auc",
    n_jobs=1   # single-threaded to save RAM
)

print("ROC-AUC scores:", scores)
print("Mean ROC-AUC:", scores.mean())
print("Std ROC-AUC:", scores.std())

# ROC-AUC scores: [0.77108981 0.77356633 0.76854147]
# Mean ROC-AUC: 0.7710658721806952
# Std ROC-AUC: 0.002051461909372355

# Ensemble

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Ensemble with soft voting
ensemble = VotingClassifier(
    estimators=[
        ("cat", cat_pipeline_tuned),
        ("lgbm", lgbm_pipeline_n),
        ("xgb", xgb_pipeline_tuned)
    ],
    voting="soft"  # soft voting = average probabilities
)





In [ ]:
# Cross-validation
cv_scores = cross_val_score(
    ensemble,
    X,
    y,
    cv=5,   # 5-fold cross-validation
    scoring="roc_auc",
    n_jobs=1
)

print("Ensemble ROC-AUC scores:", cv_scores)
print("Mean ROC-AUC:", np.mean(cv_scores))

# Mean ROC-AUC: 0.7722842373930452
# Ensemble ROC-AUC scores: [0.77355496 0.77445247 0.76765519 0.77313199 0.77262658]

In [ ]:
ensemble.fit(X, y)

print("Ensemble trained on full dataset!")

[LightGBM] [Info] Number of positive: 24825, number of negative: 282686
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.257011 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13892
[LightGBM] [Info] Number of data points in the train set: 307511, number of used features: 137
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080729 -> initscore=-2.432486
[LightGBM] [Info] Start training from score -2.432486
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [08:58:54] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Ensemble trained on full dataset!


In [ ]:
# Predict probabilities for the positive class
y_test_proba = ensemble.predict_proba(test)[:, 1]

# Predict class labels
y_test_class = ensemble.predict(test)

# Example: show first 10 predictions
print("Predicted probabilities:", y_test_proba[:10])
print("Predicted classes:", y_test_class[:10])

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Predicted probabilities: [0.07095554 0.18519606 0.02295663 0.03777164 0.12411153 0.04129492
 0.0225956  0.0399942  0.0226056  0.09952374]
Predicted classes: [0 0 0 0 0 0 0 0 0 0]


In [ ]:
import pandas as pd

# Create submission DataFrame
submission = pd.DataFrame({
    'SK_ID_CURR': test['SK_ID_CURR'],
    'TARGET': y_test_proba
})

# Save to CSV
submission.to_csv("submission.csv", index=False)

# Preview
print(submission.head())


   SK_ID_CURR    TARGET
0      100001  0.070956
1      100005  0.185196
2      100013  0.022957
3      100028  0.037772
4      100038  0.124112


In [ ]:
from google.colab import files
files.download("submission.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
submission.shape

(48744, 2)